### T(z), Up(z)

In [109]:
c = 3 * 1e10    # Скорость света
R = 0.35        # Радиус цилиндра

# температурное поле в цилиндре
def T_z(z):
    Tw, To, p = 2000, 1e4, 4
    return (Tw - To) * (z ** p) + To

# функция Планка
def up_z(z):
    return (3.084 * 1e-4) / (np.exp(4.799 * 1e4 / T_z(z)) - 1)

#### k(z) = k(T(z)) 

In [110]:
def read_data_kT(variant=1):
    data_tab = []
    with open('data/kT.txt') as file:
        line = file.readline()
        while line != '':
            line = line.split()
            if variant == 1:
                data_tab.append([int(line[1]), float(line[2])])
            else:
                data_tab.append([int(line[1]), float(line[3])])

            line = file.readline()

    return data_tab
    
import numpy as np

def getk_zFunc(variantkT):
    data_kT = read_data_kT(variantkT)
    Tvals = [np.log(data_kT[i][0]) for i in range(len(data_kT))]
    kvals = [np.log(data_kT[i][1]) for i in range(len(data_kT))]
    def k_z(z):
        T = T_z(z)
        return np.exp(np.interp([np.log(T)], Tvals, kvals)[0])
    return k_z


#### Параметры разностной схемы

In [123]:
def getStartParams(h, k_z, z0):
    M0 = 1 / (R**2 * (k_z(z0 + h) + k_z(z0)))
    K0 = -M0
    Q0 = 0
    epsilon1 = -K0 / M0
    eta1 = Q0 / M0
    print(f'M0 = {M0}, K0 = {K0}')
    print(f'epsilon1 = {epsilon1}, eta1 = {eta1}')
    return epsilon1, eta1

def vn(zn, h):
    return zn * h

def kappa_n_p_half(zn, znp1, k_z):
    return 2 / (k_z(znp1) + k_z(zn))

def kappa_n_m_half(zn, znm1, k_z):
    return 2 / (k_z(znm1) + k_z(zn))

def An(zn, h, k_z):
    return ((zn - h / 2) * kappa_n_m_half(zn, zn - h, k_z)) / (R**2 * h)

def Dn(zn, h, k_z):
    return ((zn + h / 2) * kappa_n_p_half(zn, zn + h, k_z)) / (R**2 * h)

def Bn(zn, h, k_z):
    return An(zn, h, k_z) + Dn(zn, h, k_z) - 3 * k_z(zn) * vn(zn, h)

def Fn(zn, h, k_z):
    return - 3 * k_z(zn) * up_z(zn) * vn(zn, h)

def getyN(epsilonN, etaN, h, k_z):
    # KN = - (2 - h) / (R**2 * h * ( k_z(1-h) + k_z(1) ))
    kappa = 2 / (k_z(1 - h) + k_z(1))
    MN = - (1 - h / 2) * kappa / (R**2 * h)
    KN = -MN + 3 * 0.39 / R
    QN = 0
    yN = (QN - KN * etaN) / (KN * epsilonN + MN)
    print(f'epsilonN={epsilonN}, etaN={etaN}')
    print(f'kappa={kappa}, KN={KN}, MN={MN}')
    return yN

#### Разностная схема - прогонка

In [ ]:
def epsilon_eta_ip1(zi, epsiloni, etai, h, k_z, i):
    Ai = An(zi, h, k_z)
    Bi = Bn(zi, h, k_z)
    Di = Dn(zi, h, k_z)
    Fi = Fn(zi, h, k_z)
    epsilon_ip1 = Di / (Bi - Ai * epsiloni)
    eta_ip1 = (Ai * etai + Fi) / (Bi - Ai * epsiloni)
    print(f'i={i}, zi={zi}: ', end='')
    # print(f'Ai={Ai}, Bi={Bi}, Di={Di}, Fi={Fi} ', end='')
    print(f'epsilon={epsilon_ip1}, eta={eta_ip1}')
    return (epsilon_ip1, eta_ip1)

def progonka(k_z, N=100):
    N -= 1
    z0, zN = 0, 1
    h = (zN - z0) / N
    print(f'h = {h}')
    epsilon1, eta1 = getStartParams(h, k_z, z0)
    zvals = [z0]
    epsilonVals, etaVals = [float(epsilon1)], [float(eta1)]

    for i in range(N):
        epsilonNext, etaNext = epsilon_eta_ip1(zvals[-1], epsilonVals[-1], etaVals[-1], h, k_z, i)
        epsilonVals.append(float(epsilonNext))
        etaVals.append(float(etaNext))
        zvals.append(zvals[-1] + h)

    # print(f'epsilonVals={epsilonVals}')
    # print(f'etaVals={etaVals}')

    yN = getyN(epsilonVals[-1], etaVals[-1], h, k_z)
    print(f'yN = {yN}')
    yvals = [0] * (N + 1) 
    yvals[N] = yN 
    for i in range(N - 1, -1, -1):
        yi = epsilonVals[i + 1] * yvals[i + 1] + etaVals[i + 1]
        yvals[i] = yi
    
    return yvals

def getFvals(yvals, h, k_z):
    Fvals = [0]
    for i in range(1, len(yvals) - 1):
        Fvals.append((yvals[i + 1] - yvals[i - 1]) / (2 * h))


In [160]:
import plotly.express as px
import pandas as pd
from scipy import integrate

cntSteps = 100
k_z = getk_zFunc(variantkT=1)
zvals = np.linspace(0, 1, num=cntSteps)
uvals = progonka(k_z, cntSteps)
upvals = [up_z(z) for z in zvals]

def getFvals(zvals):
    F = []
    intFunc = []
    for i in range(len(zvals)):
        intFunc.append(zvals[i] * k_z(zvals[i]) * (upvals[i] - uvals[i]))
        F.append(3 * R / zvals[i] * integrate.simpson(intFunc, zvals[:i+1]))
    return F

def get_dFvals(zvals, uvals, upvals, Fvals):
    dF = []
    for i in range(len(zvals)):
        dF.append(3 * R * k_z(zvals[i]) * (upvals[i] - uvals[i]) - Fvals[i] / zvals[i])
    return dF

Fvals = getFvals(zvals)
# dFvals = get_dFvals(zvals, uvals, upvals, Fvals)

df1 = pd.DataFrame({"x": zvals, "y": uvals, "label": ["u(z)"] * len(uvals) })
df2 = pd.DataFrame({"x": zvals, "y": upvals, "label": ["u_p(z)"] * len(upvals) })
df3 = pd.DataFrame({"x": zvals, "y": Fvals, "label": ["F(z)"] * len(Fvals) })
df4 = pd.DataFrame({"x": zvals, "y": dFvals, "label": ["dF(z)"] * len(Fvals) })
df = pd.concat([df1, df2, df3, df4]) # 
fig1 = px.line(df, x="x", y="y", color="label", title="", markers = True)
fig1.show()


h = 0.010101010101010102
M0 = 3.982080686883293, K0 = -3.982080686883293
epsilon1 = 1.0, eta1 = 0.0
i=0, zi=0: epsilon=1.0, eta=-0.0
i=1, zi=0.010101010101010102: epsilon=1.0000262636286334, eta=-6.727944337206678e-11
i=2, zi=0.020202020202020204: epsilon=1.0000472758796697, eta=-1.211064027641202e-10
i=3, zi=0.030303030303030304: epsilon=1.000067539378684, eta=-1.730150706458814e-10
i=4, zi=0.04040404040404041: epsilon=1.0000875545867, eta=-2.2428710513663378e-10
i=5, zi=0.05050505050505051: epsilon=1.0001074578742868, eta=-2.752711903975484e-10
i=6, zi=0.060606060606060615: epsilon=1.000127301300409, eta=-3.26099663851434e-10
i=7, zi=0.07070707070707072: epsilon=1.0001471085050215, eta=-3.768316149089382e-10
i=8, zi=0.08080808080808081: epsilon=1.0001668910923218, eta=-4.2749477303380224e-10
i=9, zi=0.09090909090909091: epsilon=1.00018665461682, eta=-4.781007618781866e-10
i=10, zi=0.10101010101010101: epsilon=1.000206401103748, eta=-5.286514762025127e-10
i=11, zi=0.1111111111111111: 

C:\Users\ck4te\AppData\Local\Temp\ipykernel_23124\3508861816.py:16: RuntimeWarning:

divide by zero encountered in scalar divide

C:\Users\ck4te\AppData\Local\Temp\ipykernel_23124\3508861816.py:16: RuntimeWarning:

invalid value encountered in scalar multiply



In [161]:


# zvals = np.linspace(0, 1, num=cntSteps)




def IntegrF(z, u : list[float]):
    k = getkFunc()
    F = [0] * len(u)
    intFunc = []
    for i in range(len(u)):
        intFunc.append(z[i] * k(T(z[i])) * (Plank(z[i])) - u[i])
        F[i] = 3*R/z[i] * sp.integrate.simpson(intFunc, z[:i+1])
    return F
